This is to try and reproduce logs from Run 29.

In [2]:
%matplotlib inline
import pandas as pd
import pandana as pdna
from urbansim.utils import misc, networks

store = pd.HDFStore('./../runs/run29.h5')
pd.options.mode.chained_assignment = None

/home/pksohn/.conda/envs/semcog_urbansim/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
year = 2035

In [4]:
year_tables = filter(lambda name: str(year) in name, store.keys())
for y in year_tables:
    print y

/2035/buildings
/2035/dropped_buildings
/2035/feasibility
/2035/households
/2035/jobs
/2035/parcels
/2035/persons


In [18]:
parcel_lids = store['{}/parcels'.format(year)][['large_area_id']]

NameError: name 'year' is not defined

In [5]:
dropped_buildings, feasibility, households, jobs, parcels, persons = (
    store[table] for table in year_tables if table.split('/')[-1] != 'buildings'
)

# NEED PREVIOUS YEAR BUILDINGS!
buildings = store['{}/buildings'.format(year-1)]

In [6]:
# Set up walk network

with pd.HDFStore('./../data/semcog_networks.h5') as net_store:
    nodes = net_store['nodes_mgf14_ext_walk']
    edges = net_store['edges_mgf14_ext_walk']
    net = pdna.Network(nodes["x"], nodes["y"], 
                       edges["from"], edges["to"],
                       edges[['feet']])

In [7]:
# Focus on a large area
lid = 93

In [7]:
# Calculate the "raw" feasibility table for residential (with NAs)
# Filter for NAs

residential_raw = (feasibility['residential']
               .merge(parcel_lids, left_index=True, right_index=True))
# residential_raw = residential_raw.loc[residential_raw.large_area_id == lid]
print(len(residential_raw))
# MATCHES LOG

df = residential_raw.loc[residential_raw.max_profit_far > 0]
print(len(df))
# DOESN'T MATCH ANYTHING YET BUT THAT IS EXPECTED

# LOOKING FOR 184 FEASIBLE NET UNITS

NameError: name 'feasibility' is not defined

In [9]:
df.head()

,non_residential_sqft,parking_config,financing_cost,building_sqft,stories,total_cost,max_profit,building_cost,max_profit_far,construction_time,building_revenue,parking_ratio,residential_sqft,large_area_id
parcel_id,,,,,,,,,,,,,,
7005248,0.0,surface,3424.50612,857.52,1.0,101267.53812,24447.135292,97843.032,0.01,12.0,125714.673412,0.0,728.892,93
7006027,0.0,surface,5840.57362,1462.52,1.0,172714.10562,66816.174062,166873.532,0.04,12.0,239530.279682,0.0,1243.142,93
7006463,0.0,surface,4919.67252,1231.92,1.0,145481.74452,101330.636035,140562.072,0.04,12.0,246812.380555,0.0,1047.132,93
7006500,0.0,surface,4792.35974,1200.04,1.0,141716.92374,98708.371053,136924.564,0.04,12.0,240425.294793,0.0,1020.034,93
7006501,0.0,surface,4801.30518,1202.28,1.0,141981.45318,111422.245326,137180.148,0.04,12.0,253403.698506,0.0,1021.938,93


In [10]:
parcels.head()

,nodeid_drv,zone_id,sev_value,city_id,land_use_type_id,census_bg_id,large_area_id,semmcd,centroid_y,centroid_x,county_id,nodeid_walk,pptytax,school_id,parcel_sqft
parcel_id,,,,,,,,,,,,,,,
8000001,21850,184,55094700,508,2,5172001,5,5,304988.032895,1.348211e+07,163,175775,69.1307,82010,566295
8000003,951,189,1474000,508,6,5172001,5,5,304568.385328,1.348158e+07,163,133697,69.1307,82010,24811
8000010,20171,189,33100,508,20,5172001,5,5,304781.519606,1.348179e+07,163,74087,69.1307,82010,728
8000012,17737,189,0,508,1,5172001,5,5,304992.893630,1.348153e+07,163,247560,69.1307,82010,6301
8000019,34215,185,822800,508,3,5172001,5,5,306014.316625,1.348178e+07,163,10997,69.1307,82010,17192


In [11]:
# Get ave_unit_sqft
nodes_walk = pd.DataFrame(index=net.node_ids)
b = buildings.loc[buildings.sqft_per_unit > 0]
b = b.merge(parcels[['nodeid_walk']], left_on='parcel_id', right_index=True)
net.set(b['nodeid_walk'], variable=b['sqft_per_unit'])
nodes_walk['ave_unit_sqft'] = net.aggregate(500, type='average', decay='flat')
ave_unit_size = misc.reindex(nodes_walk.ave_unit_sqft, parcels.nodeid_walk)
ave_unit_size[ave_unit_size < 1000] = 1000
df['ave_unit_size'] = ave_unit_size

In [12]:
df['parcel_size'] = parcels.parcel_sqft

In [13]:
df['current_units'] = (buildings.residential_units
                                .groupby(buildings.parcel_id).sum()
                                .reindex(parcels.index).fillna(0))

In [14]:
df['residential_units'] = (df.residential_sqft
                           / df.ave_unit_size).round()

In [15]:
df['net_units'] = df.residential_units - df.current_units

In [16]:
new_df = df.loc[df.net_units > 0]
print(len(new_df))

21


In [17]:
print(new_df.net_units.sum())

84.0


In [18]:
# Why so few profitable buildings?
df

,non_residential_sqft,parking_config,financing_cost,building_sqft,stories,total_cost,max_profit,building_cost,max_profit_far,construction_time,building_revenue,parking_ratio,residential_sqft,large_area_id,ave_unit_size,parcel_size,current_units,residential_units,net_units
parcel_id,,,,,,,,,,,,,,,,,,,
7005248,0.0,surface,3424.506120,857.520,1.0,1.012675e+05,24447.135292,97843.0320,0.010,12.0,1.257147e+05,0.0,728.89200,93,1913.583374,85752,0.0,0.0,0.0
7006027,0.0,surface,5840.573620,1462.520,1.0,1.727141e+05,66816.174062,166873.5320,0.040,12.0,2.395303e+05,0.0,1243.14200,93,2902.600098,36563,0.0,0.0,0.0
7006463,0.0,surface,4919.672520,1231.920,1.0,1.454817e+05,101330.636035,140562.0720,0.040,12.0,2.468124e+05,0.0,1047.13200,93,2647.642822,30798,0.0,0.0,0.0
7006500,0.0,surface,4792.359740,1200.040,1.0,1.417169e+05,98708.371053,136924.5640,0.040,12.0,2.404253e+05,0.0,1020.03400,93,2647.642822,30001,0.0,0.0,0.0
7006501,0.0,surface,4801.305180,1202.280,1.0,1.419815e+05,111422.245326,137180.1480,0.040,12.0,2.534037e+05,0.0,1021.93800,93,2912.500000,30057,0.0,0.0,0.0
7006506,0.0,surface,5637.703820,1411.720,1.0,1.667150e+05,130832.262178,161077.2520,0.040,12.0,2.975472e+05,0.0,1199.96200,93,2912.500000,35293,1.0,0.0,-1.0
7006507,0.0,surface,5322.377060,1332.760,1.0,1.573903e+05,123514.582028,152067.9160,0.040,12.0,2.809049e+05,0.0,1132.84600,93,2912.500000,33319,0.0,0.0,0.0
7006508,0.0,surface,4792.998700,1200.200,1.0,1.417358e+05,111229.479689,136942.8200,0.040,12.0,2.529653e+05,0.0,1020.17000,93,2912.500000,30005,0.0,0.0,0.0
7006523,0.0,surface,5239.312260,1311.960,1.0,1.549339e+05,3942.681108,149694.6360,0.040,12.0,1.588766e+05,0.0,1115.16600,93,2250.133301,32799,1.0,0.0,-1.0


In [19]:
df.net_units.describe()

count    1318.000000
mean       -0.220030
std         1.291345
min        -1.000000
25%        -1.000000
50%         0.000000
75%         0.000000
max        41.000000
Name: net_units, dtype: float64

### Look at prices within a large area
Are the low-priced neighborhoods ever getting developed?

In [8]:
# lid = 3
p = store['base/parcels']
# p = p.loc[p.large_area_id == lid]

b = store['base/buildings']
b = b.merge(p[['large_area_id']], left_on='parcel_id', right_index=True)
# b = b.loc[b.large_area_id == lid]

In [9]:
# Get average prices in neighborhood in base year

nodes_walk = pd.DataFrame(index=net.node_ids)
bldg = b.loc[(81 <= b.building_type_id)
            & (b.building_type_id <= 84)
            & (0 < b.sqft_price_res)
            & (b.sqft_price_res < 500)]
bldg = bldg.merge(p[['nodeid_walk']], left_on='parcel_id', right_index=True)

net.set(bldg['nodeid_walk'], variable=bldg['sqft_price_res'])
nodes_walk['sqft_price_res'] = net.aggregate(2000, type='average', decay='flat')
res_prices = misc.reindex(nodes_walk.sqft_price_res, p.nodeid_walk)
p['res_prices'] = res_prices

In [13]:
p['res_decile'] = pd.qcut(p['res_prices'], 10, labels=False)

In [20]:
for lid in p.large_area_id.unique():
    deciles = pd.DataFrame(index=p.res_decile.unique())
    for year in range(2016, 2046):
        buildings_name = '{}/buildings'.format(year)
        buildings = store[buildings_name]
        buildings = buildings[['year_built', 'parcel_id']]
        buildings = buildings.merge(p[['res_prices', 'res_decile', 'large_area_id']],
                                    left_on='parcel_id', right_index=True)
        buildings = buildings.loc[buildings.large_area_id == lid]
        buildings = buildings.loc[buildings.year_built == year]
        built = buildings.groupby('res_decile').count().year_built
        deciles[year] = built
    result = deciles.fillna(0).sort_index()
    result.to_csv('run29_allbuildings_by_resdeciles_lid{}.csv'.format(lid))

In [46]:
df = pd.read_csv('run29_allbuildings_by_resdeciles_lid93.csv', index_col=0)

In [47]:
df[[y for y in df.columns if int(y) > 2035]]

,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045
0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0
5,1.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,2.0,0.0
6,1.0,2.0,3.0,5.0,6.0,7.0,1.0,6.0,4.0,8.0
7,3.0,2.0,2.0,2.0,278.0,2.0,4.0,8.0,9.0,3.0
8,0.0,2.0,8.0,5.0,4.0,2.0,7.0,8.0,4.0,5.0
9,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0
